# Lesson 3 – Sequential Pattern: Bug Report Cleaning & Triage Pipeline

This notebook is part of the **LangGraph Agentic AI – Intro Course**.

In this lesson, you will build a **sequential multi-node LangGraph pipeline** that takes a raw bug report, cleans it, extracts metadata, and produces a simple triage recommendation.


## 1. Objectives & Prerequisites

**Objectives**

By the end of this lesson, you can:

- Design a `TypedDict` state for a bug report processing pipeline.
- Implement **multiple nodes** that each perform one step of the pipeline.
- Wire nodes together in a **sequential LangGraph** (A → B → C).
- Inspect the state after each step to understand how data flows through the graph.

**Prerequisites**

- Lesson 1: single-node pattern.
- Lesson 2: multiple inputs state.
- Comfortable with Python strings and basic `if`/`elif`/`else` logic.


## 2. Environment Setup

Make sure you have installed the course dependencies (from the repo root):

```bash
python -m venv .venv
source .venv/bin/activate   # on Windows: .venv\Scripts\activate
pip install -r env/requirements.txt
```

Then start Jupyter and select the `langgraph-intro` (or equivalent) Python kernel.


## 3. Concept Warm-up

In the previous lessons, we built agents with a **single node**. Real workflows often need **multiple steps**:

1. Clean incoming data.
2. Extract structured fields.
3. Make a decision or recommendation.

In this lesson, we'll process a bug report like:

> "App keeps freezing when I click Export with CSV on Mac. Urgent!!!"

Our pipeline will:

1. Clean the text (normalize whitespace).
2. Extract fields:
   - `severity` (low/medium/high)
   - `platform` (web/mac/windows/unknown)
   - `feature` (export/login/unknown)
3. Generate a triage recommendation string.


### Scratch cell (optional)

Use this cell for quick string experiments while you follow along.


In [ ]:
# Scratch space for experimentation

text = "App keeps freezing when I click Export with CSV on Mac. Urgent!!!"
text.lower()

## 4. Define the State

We'll use a `TypedDict` called `BugState` with the following fields:

- `raw_report: str` – the original user-submitted bug report.
- `clean_report: str` – cleaned/normalized text.
- `severity: str` – e.g. "low", "medium", "high".
- `platform: str` – e.g. "web", "mac", "windows", "unknown".
- `feature: str` – e.g. "export", "login", "unknown".
- `recommendation: str` – final triage string.


In [ ]:
from typing import TypedDict

class BugState(TypedDict):
    raw_report: str
    clean_report: str
    severity: str
    platform: str
    feature: str
    recommendation: str

# Example initial state
initial_state: BugState = {
    "raw_report": "App keeps freezing when I click Export with CSV on Mac. Urgent!!!",
    "clean_report": "",
    "severity": "",
    "platform": "",
    "feature": "",
    "recommendation": "",
}

initial_state

## 5. Node 1 – Cleaning the Report

First node: normalize whitespace, strip leading/trailing spaces, and store the result in `clean_report`.

We won't do heavy text processing here; the goal is to show a simple first step in a pipeline.


In [ ]:
def clean_report_node(state: BugState) -> BugState:
    """Clean and normalize the raw bug report text.

    - Collapse multiple spaces into one
    - Strip leading/trailing spaces
    """
    raw = state["raw_report"]
    # Split and re-join collapses consecutive whitespace
    clean = " ".join(raw.split())
    clean = clean.strip()
    state["clean_report"] = clean
    return state


# Quick local test
test_state = initial_state.copy()
clean_report_node(test_state)

## 6. Node 2 – Metadata Extraction

Second node: look at `clean_report` and extract:

- `severity` – based on certain keywords.
- `platform` – infer from words like "mac", "windows", "browser".
- `feature` – infer from words like "export", "login".


In [ ]:
def extract_metadata_node(state: BugState) -> BugState:
    """Extract severity, platform, and feature from the clean report.

    This is a very simple rule-based approach for teaching purposes.
    """
    text = state["clean_report"].lower()

    # severity
    if any(word in text for word in ["urgent", "crash", "freez"]):
        severity = "high"
    else:
        severity = "medium"

    # platform
    if "mac" in text:
        platform = "mac"
    elif "windows" in text:
        platform = "windows"
    elif "web" in text or "browser" in text:
        platform = "web"
    else:
        platform = "unknown"

    # feature
    if "export" in text:
        feature = "export"
    elif "login" in text:
        feature = "login"
    else:
        feature = "unknown"

    state["severity"] = severity
    state["platform"] = platform
    state["feature"] = feature
    return state


# Quick local test
test_state2 = initial_state.copy()
clean_report_node(test_state2)
extract_metadata_node(test_state2)

## 7. Node 3 – Recommendation Generation

Third node: use `severity`, `platform`, and `feature` to generate a simple triage recommendation.

Example:

> "High severity bug on mac in export. Assign to on-call engineer immediately."


In [ ]:
def make_recommendation_node(state: BugState) -> BugState:
    """Create a simple triage recommendation based on extracted metadata."""
    severity = state["severity"] or "unknown"
    platform = state["platform"] or "unknown"
    feature = state["feature"] or "unknown"

    recommendation = (
        f"{severity.title()} severity bug on {platform} in {feature}."
    )

    if severity == "high":
        recommendation += " Assign to on-call engineer immediately."
    else:
        recommendation += " Add to regular sprint backlog."

    state["recommendation"] = recommendation
    return state


# Quick local test
test_state3 = initial_state.copy()
clean_report_node(test_state3)
extract_metadata_node(test_state3)
make_recommendation_node(test_state3)

## 8. Build the Sequential LangGraph Pipeline

Now let's wire these three nodes into a `StateGraph`:

1. Entry: `clean_report_node`
2. Then: `extract_metadata_node`
3. Then: `make_recommendation_node`
4. Finish at `make_recommendation_node`


In [ ]:
from langgraph.graph import StateGraph

# 1. Create the graph
graph = StateGraph(BugState)

# 2. Add nodes
graph.add_node("clean_report", clean_report_node)
graph.add_node("extract_metadata", extract_metadata_node)
graph.add_node("make_recommendation", make_recommendation_node)

# 3. Set the entry point
graph.set_entry_point("clean_report")

# 4. Add edges for a sequential pipeline
graph.add_edge("clean_report", "extract_metadata")
graph.add_edge("extract_metadata", "make_recommendation")

# 5. Set the finish point
graph.set_finish_point("make_recommendation")

# 6. Compile the graph
app = graph.compile()

# 7. Invoke the graph
result = app.invoke(initial_state)
result

Try changing `raw_report` in `initial_state` and re-running the pipeline.

Examples:

- Include words like "login" instead of "export".
- Remove words like "urgent" and see how `severity` changes.
- Omit platform keywords like "mac" and see `platform == "unknown"`.


## 9. Exercises

Use the cells below to implement the exercises described for this lesson.

---

### Exercise 1 – More Severity Rules

**Goal:**

Extend the severity logic so that:

- If the text contains `"data loss"`, treat it as `"high"` severity.
- If the text contains `"typo"` and **no** high-severity keywords, treat it as `"low"` severity.

Update `extract_metadata_node` accordingly.


In [ ]:
# TODO: Extend severity logic in extract_metadata_node.

# Hint:
# if "data loss" in text:
#     severity = "high"
# elif "typo" in text and not any(word in text for word in ["urgent", "crash", "freez"]):
#     severity = "low"


### Exercise 2 – Unknown Handling

**Goal:**

If either `platform == "unknown"` or `feature == "unknown"`, append this note to the recommendation:

> " Need more info on platform/feature."

Update `make_recommendation_node` to add this message when needed.


In [ ]:
# TODO: Update make_recommendation_node to append a note when platform/feature is unknown.

# Hint:
# if platform == "unknown" or feature == "unknown":
#     recommendation += " Need more info on platform/feature."


### Stretch Exercise 1 – Third-party Flag

**Goal:**

Add a new field to the state:

```python
is_third_party: bool
```

Set `is_third_party = True` if the text contains words like `"plugin"` or `"extension"`, otherwise `False`.

Then, update the recommendation:

- If `is_third_party` is `True`, append:  
  > " Route to integrations team."


In [ ]:
# TODO: Extend BugState and update extract_metadata_node + make_recommendation_node for is_third_party.

# Hint:
# if any(w in text for w in ["plugin", "extension"]):
#     state["is_third_party"] = True
# else:
#     state["is_third_party"] = False


### Stretch Exercise 2 – Assign Team Node

**Goal:**

Add a 4th node:

```python
assign_team_node
```

This node should:

- Look at `feature` and/or `platform`.
- Set a new field in the state:

  ```python
  assigned_team: str
  ```

For example:

- `feature == "export"` → `assigned_team = "Data Export Team"`
- `feature == "login"` → `assigned_team = "Auth Team"`
- Otherwise → `assigned_team = "General Backend Team"`

Then:

- Add the node to the graph after `make_recommendation_node`.
- Update the finish point to `assign_team_node`.


In [ ]:
# TODO: Implement assign_team_node and add it to the sequential pipeline.

# Hint:
# def assign_team_node(state: BugState) -> BugState:
#     feature = state["feature"]
#     if feature == "export":
#         team = "Data Export Team"
#     elif feature == "login":
#         team = "Auth Team"
#     else:
#         team = "General Backend Team"
#     state["assigned_team"] = team
#     return state


---

You’ve now completed the scaffold for **Lesson 3 – Sequential Pattern**.

Next steps:

- Commit this notebook and your solutions to your Git repo.
- Then proceed to Lesson 4, where you’ll build a **conditional router agent** for customer support messages.
